In [156]:
import sqlite3
import copy
from typing import (
    Any,
    Generic,
    TypeVar,
    Self,
    Generator,
    Type
)
import uuid

# ORM SOURCE CODE

In [157]:


DATATYPES: dict[str, str] = {
    'int':'INTEGER',
    'str':'TEXT',
    'float':'REAL',
    'bytes':'BLOB'
}

T = TypeVar('T')

class SQLModel:
    
    def __init__(self,**kwargs) -> None:
        self.__annotations__ = vars(self.__class__)['__annotations__']
        self.__model_fields__: dict[str, Any] = {key:copy.deepcopy(getattr(self,key)) for key in self.__annotations__.keys()}
        self.__name__: str = self.__class__.__name__
        self.__sql__ = f"CREATE TABLE {self.__name__.lower()} ({', '.join([self.__model_fields__[x].__sql__ for x in self.__model_fields__.keys()])});"
        for key,val in kwargs.items():
            setattr(self,key,val)
    
    def __repr__(self) -> str:
        data: list[str] = [f'{key}={getattr(self,key)}' for key in self.__annotations__.keys()]
        return f"{self.__class__.__name__}({','.join(data)})"
    
    @staticmethod    
    def execute_table(conn:sqlite3.Connection) -> sqlite3.Cursor:
        
        #create cursor
        cursor: sqlite3.Cursor = conn.cursor()
        
        #tables
        sql_create_tables:list[str] = [x().__sql__ for x in SQLModel.__subclasses__()]
        
        script:str = f"BEGIN; {' '.join(sql_create_tables)} COMMIT;"
        
        return cursor.executescript(script)
class Field(Generic[T]):
    
    def __init__(
        self,
        primary_key=False,
        unique=False,
        required=False,
        auto_increment=False,
        name='',
        _type:Any=None,
    ) -> None:
        
        self.__name__: str = name
        self.__type__: type = _type
        
        self.__constraints__:list[str] = []
        
        if primary_key:
            self.__constraints__.append("PRIMARY KEY")
            
        if unique:
            self.__constraints__.append("UNIQUE")
            
        if required:
            self.__constraints__.append("NOT NULL")
            
        if auto_increment:
            self.__constraints__.append("AUTOINCREMENT")
        
        self.__sql__:str = f"{self.__name__} {DATATYPES[self.__type__.__name__]} {' '.join(self.__constraints__)}"
    
    def __repr__(self) -> str:
        return self.__name__
    
    def __str__(self) -> str:
        return ""
    
    def __eq__(self, __value: object) -> str:
        if isinstance(__value,str):
            return f"{self.__name__} = '{__value}'"
        else:
            return f"{self.__name__} = '{__value}'"

class ForeignKey(Field):
    
    def __init__(self,name:str,table:Type[SQLModel],_type:type,column:str) -> None:
        self.__name__ = name
        self.__type__ = _type
        self.__sql__ = f"{self.__name__} {DATATYPES[self.__type__.__name__]} , FOREIGN KEY ({self.__name__}) REFERENCES {table().__name__}({column})"
        pass
    
class Query:
    
    def __init__(self,table:Type[SQLModel]) -> None:
        self.table = table
        self.__query__:list[str] = []
        
    def select(self,*columns:Field|ForeignKey) -> Self:
        
        select_syntax:str = "SELECT {columns} from {table}"
        
        if len(columns) == 0:
            
            self.__query__.append(
                select_syntax.format(
                    table=self.table.__name__.lower(),
                    columns=", ".join([f'{self.table.__name__.lower()}.{x}' for x in self.table().__model_fields__.keys()])
                )
            )
            
            return self
        
        self.__query__.append(
            select_syntax.format(
                    table=self.table.__name__.lower(),
                    columns=", ".join([x.__name__ for x in columns])
                )
        )
        
        return self
    
    def where(self,*conditions) -> Self:
        
        where_syntax: str = "WHERE {conditions}"
        
        self.__query__.append(
            where_syntax.format(
                conditions="".join(conditions)
            )
        )
                
        return self
    
    def insert(self,values_obj) -> Self:
        insert_syntax: str = "INSERT INTO {table} VALUES ({values})"
        
        values_obj_str:list[str] = []
        
        for val in [str(x) for x in {key:val for key,val in values_obj.__dict__.items() if '__' not in key}.values()]:
            if isinstance(val,str):
                values_obj_str.append(f"'{val}'")
            else:
                values_obj_str.append(val)
        
        self.__query__.append(
            insert_syntax.format(
                values=', '.join(values_obj_str),
                table=self.table.__name__.lower()
            )
        )
        return self
    
class Session:
    
    def __init__(self,engine:sqlite3.Connection) -> None:
        self.engine: sqlite3.Connection = engine
        self.cursor: sqlite3.Cursor = engine.cursor()
    
    def __enter__(self) -> Self:
        return self
    
    def __exit__(self,exception_type, exception_value, exception_traceback) -> None:        
        self.cursor.close()
        if exception_value:
            raise Exception(exception_value)
        pass
    
    def execute(self,query_obj:Query) -> Self:
        self.cursor.execute(' '.join(query_obj.__query__))
        return self
    
    def commit(self) -> Self:
        self.engine.commit()
        return self
    
    def fetchall(self) -> list:
        columns:tuple = tuple([x[0] for x in self.cursor.description])
        rows_data = self.cursor.fetchall()
        data:list[dict] = []
        for single_row in rows_data:
            row_value:dict = {}
            for value,col in zip(single_row,columns):
                row_value.update({col:value})
            data.append(row_value)
        return data

# ORM CODE FLOW

In [158]:
class User(SQLModel):
    id:Field[str] = Field(name='id',_type=str,primary_key=True)
    name:Field[str] = Field(name='name',_type=str,required=True)
    email:Field[str] = Field(name='email',_type=str,required=True)

class Item(SQLModel):
    id:Field[str] = Field(name='id',_type=str,primary_key=True)
    name:Field[str] = Field(name='name',_type=str,required=True)
    price:Field[float] = Field(name='price',_type=float,required=True)
    user_id:ForeignKey = ForeignKey(name='user_id',_type=int,table=User,column="id")

In [159]:
conn: sqlite3.Connection = sqlite3.connect(":memory:")
SQLModel.execute_table(conn)

In [160]:
user_var1 = User(id=str(uuid.uuid1()),name="karl",email="karlalferezfx@gmail.com")
karl_item_1 = Item(id=str(uuid.uuid1()),name="soap",price=3.14,user_id=user_var1.id)
karl_item_2 = Item(id=str(uuid.uuid1()),name="shampoo",price=1.50,user_id=user_var1.id)
karl_item_3 = Item(id=str(uuid.uuid1()),name="toothpaste",price=5.00,user_id=user_var1.id)

user_var2 = User(id=str(uuid.uuid1()),name="andrei",email="andrei_barlaan@gmail.com")
andrei_item_1 = Item(id=str(uuid.uuid1()),name="face mask",price=0.01,user_id=user_var2.id)
andrei_item_2 = Item(id=str(uuid.uuid1()),name="yosi",price=0.10,user_id=user_var2.id)
andrei_item_3 = Item(id=str(uuid.uuid1()),name="disposable vape",price=5.50,user_id=user_var2.id)

In [161]:
with Session(conn) as db:
  
  insert_user1: Query = Query(User).insert(user_var1)
  insert_user2: Query = Query(User).insert(user_var2)
  db.execute(insert_user1)
  db.execute(insert_user2)
  db.commit()

  for item in [karl_item_1,karl_item_2,karl_item_3,andrei_item_1,andrei_item_2,andrei_item_3]:
    insert_stmt:Query = Query(Item).insert(item)
    db.execute(insert_stmt)
  db.commit()

  select_stmt: Query = Query(Item).select().where(Item.user_id == user_var2.id)
  
  data: list = db.execute(select_stmt).fetchall()

In [162]:
data:list[Item] = [Item(**x) for x in data]
print(data)

[Item(id=e25f4c8b-681c-11ee-9179-44032cfb3c03,name=face mask,price=0.1,user_id=e25f25ee-681c-11ee-80bc-44032cfb3c03), Item(id=e25f4c8c-681c-11ee-8b2f-44032cfb3c03,name=cuddles,price=1.0,user_id=e25f25ee-681c-11ee-80bc-44032cfb3c03), Item(id=e25f4c8d-681c-11ee-9259-44032cfb3c03,name=ice cream,price=1.5,user_id=e25f25ee-681c-11ee-80bc-44032cfb3c03)]
